# Simple Linear Transformation

## BLinear-v1

In [1]:
import argparse
from types import SimpleNamespace
import sys
sys.path.append('/data/lige/HKN')# Please change accordingly!

from __future__ import division
from __future__ import print_function

from geoopt import ManifoldParameter as geoopt_ManifoldParameter
from manifolds.base import ManifoldParameter as base_ManifoldParameter

import datetime
import json
import logging
from optim import RiemannianAdam, RiemannianSGD
import os
import pickle
import time

import numpy as np
import torch
from config import parser
from models.base_models import NCModel, LPModel, GCModel
from utils.data_utils import load_data, get_nei, GCDataset, split_batch
from utils.train_utils import get_dir_name, format_metrics
from utils.eval_utils import acc_f1

from geoopt import ManifoldParameter as geoopt_ManifoldParameter
from manifolds.base import ManifoldParameter as base_ManifoldParameter

import torch.nn.functional as F


config_args = {
    'training_config': {
        'lr': (1e-4, 'learning rate'),
        'dropout': (0.25, 'dropout probability'),
        'cuda': (0, 'which cuda device to use (-1 for cpu training)'),
        'epochs': (1000, 'maximum number of epochs to train for'),
        'weight_decay': (1e-3, 'l2 regularization strength'),
        'optimizer': ('radam', 'which optimizer to use, can be any of [rsgd, radam]'),
        'momentum': (0.999, 'momentum in optimizer'),
        'patience': (15, 'patience for early stopping'),
        'seed': (1234, 'seed for training'),
        'log_freq': (5, 'how often to compute print train/val metrics (in epochs)'),
        'eval_freq': (1, 'how often to compute val metrics (in epochs)'),
        'save': (0, '1 to save model and logs and 0 otherwise'),
        'save_dir': (None, 'path to save training logs and model weights (defaults to logs/task/date/run/)'),
        'sweep_c': (0, ''),
        'lr_reduce_freq': (None, 'reduce lr every lr-reduce-freq or None to keep lr constant'),
        'gamma': (0.5, 'gamma for lr scheduler'),
        'print_epoch': (True, ''),
        'grad_clip': (None, 'max norm for gradient clipping, or None for no gradient clipping'),
        'min_epochs': (300, 'do not early stop before min-epochs')
    },
    'model_config': {
        'use_geoopt': (False, "which manifold class to use, if false then use basd.manifold"),
        'AggKlein':(False, "if false, then use hyperboloid centorid for aggregation"),
        'corr': (0,'0: d(x_i ominus x, x_k), 1: d(x_ik,x_k)'),
        'task': ('nc', 'which tasks to train on, can be any of [lp, nc]'),
        'model': ('BMLP', 'which encoder to use, can be any of [Shallow, MLP, HNN, GCN, GAT, HyperGCN, HyboNet,BKNet,BMLP]'),
        'dim': (64, 'embedding dimension'),
        'manifold': ('PoincareBall', 'which manifold to use, can be any of [Euclidean, Hyperboloid, PoincareBall, Lorentz]'),
        'c': (1.0, 'hyperbolic radius, set to None for trainable curvature'),
        'r': (2., 'fermi-dirac decoder parameter for lp'),
        't': (1., 'fermi-dirac decoder parameter for lp'),
        'margin': (2., 'margin of MarginLoss'),
        'pretrained_embeddings': (None, 'path to pretrained embeddings (.npy file) for Shallow node classification'),
        'pos_weight': (0, 'whether to upweight positive class in node classification tasks'),
        'num_layers': (2, 'number of hidden layers in encoder'),
        'bias': (1, 'whether to use bias (1) or not (0)'),
        'act': ('relu', 'which activation function to use (or None for no activation)'),
        'n_heads': (4, 'number of attention heads for graph attention networks, must be a divisor dim'),
        'alpha': (0.2, 'alpha for leakyrelu in graph attention networks'),
        'double_precision': ('1', 'whether to use double precision'),
        'use_att': (0, 'whether to use hyperbolic attention or not'),
        'local_agg': (0, 'whether to local tangent space aggregation or not'),
        'kernel_size': (8, 'number of kernels'),
        'KP_extent': (0.66, 'influence radius of each kernel point'),
        'radius': (1, 'radius used for kernel point init'),
        'deformable': (False, 'deformable kernel'),
        'linear_before': (64, 'dim of linear before gcn')#64
    },
    'data_config': {
        'dataset': ('wisconsin', 'which dataset to use(cornell,wisconsin,squirrel,cora)'),
        'batch_size': (32, 'batch size for gc'),
        'val_prop': (0.05, 'proportion of validation edges for link prediction'),
        'test_prop': (0.1, 'proportion of test edges for link prediction'),
        'use_feats': (1, 'whether to use node features or not'),
        'normalize_feats': (1, 'whether to normalize input node features'),
        'normalize_adj': (1, 'whether to row-normalize the adjacency matrix'),
        'split_seed': (1234, 'seed for data splits (train/test/val)'),
        'split_graph': (False, 'whether to split the graph')
    }
}

# 将所有参数转换为 SimpleNamespace
args = SimpleNamespace(
    **{k: v[0] for config in config_args.values() for k, v in config.items()}
)

#choose which manifold class to follow 
if args.use_geoopt == False:
    ManifoldParameter = base_ManifoldParameter
else:
    ManifoldParameter = geoopt_ManifoldParameter
np.random.seed(args.seed)#args.seed
torch.manual_seed(args.seed)#args.seed
if int(args.cuda):#args.double_precision
    torch.set_default_dtype(torch.float64)
if int(args.cuda) >= 0:#args.cuda
    torch.cuda.manual_seed(args.seed)#args.seed
args.device = 'cuda:' + str(args.cuda) if int(args.cuda) >= 0 else 'cpu' #args.device actually,<-args.cuda
args.patience = args.epochs if not args.patience else args.patience #args.patience<-args.epochs|args.patience

print(f'Using: {args.device}')
print("Using seed {}.".format(args.seed))
print(f"Dataset: {args.dataset}")

# Load data
data = load_data(args, os.path.join('data', args.dataset))
if args.task == 'gc':
    args.n_nodes, args.feat_dim = data['features'][0].shape
else:
    args.n_nodes, args.feat_dim = data['features'].shape
if args.task == 'nc':
    Model = NCModel
    args.n_classes = int(data['labels'].max() + 1)
    args.data = data
    print(f'Num classes: {args.n_classes}')
elif args.task == 'gc':
    Model = GCModel
    args.n_classes = int(data['labels'].max() + 1)
    print(f'Num classes: {args.n_classes}')
else:
    args.nb_false_edges = len(data['train_edges_false'])
    args.nb_edges = len(data['train_edges'])
    if args.task == 'lp':
        Model = LPModel
        args.n_classes = 2

if not args.lr_reduce_freq:
    args.lr_reduce_freq = args.epochs


###A simple check on data
print(data.keys())
print(data['adj_train'].todense().shape)
print(data['features'].shape)
###A simple check on data

# Model and optimizer
model = Model(args)
print(str(model))
no_decay = ['bias', 'scale']
optimizer_grouped_parameters = [{
    'params': [
        p for n, p in model.named_parameters()
        if p.requires_grad and not any(
            nd in n
            for nd in no_decay) and not isinstance(p, ManifoldParameter)
    ],
    'weight_decay':
    args.weight_decay
}, {
    'params': [
        p for n, p in model.named_parameters() if p.requires_grad and any(
            nd in n
            for nd in no_decay) or isinstance(p, ManifoldParameter)
    ],
    'weight_decay':
    0.0
}]
if args.optimizer == 'radam':
    optimizer = RiemannianAdam(params=optimizer_grouped_parameters,
                                lr=args.lr,
                                stabilize=10)
elif args.optimizer == 'rsgd':
    optimizer = RiemannianSGD(params=optimizer_grouped_parameters,
                                lr=args.lr,
                                stabilize=10)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                step_size=int(
                                                args.lr_reduce_freq),
                                                gamma=float(args.gamma))
tot_params = sum([np.prod(p.size()) for p in model.parameters()])
model = model.to(args.device)
for x, val in data.items():
    if torch.is_tensor(data[x]):
        data[x] = data[x].to(args.device)
print(f"Total number of parameters: {tot_params}")

# Train model for nc:
t_total = time.time()
counter = 0
best_val_metrics = model.init_metric_dict()
best_test_metrics = None
best_emb = None
if args.n_classes > 2:
    f1_average = 'micro'
else:
    f1_average = 'binary'

if args.model == 'HKPNet':
    nei, nei_mask = get_nei(data['adj_train'])
    nei = nei.to(args.device)
    nei_mask = nei_mask.to(args.device)
elif args.model == 'BKNet':
    nei, nei_mask = get_nei(data['adj_train'])
    nei = nei.to(args.device)
    nei_mask = nei_mask.to(args.device) #nei/nei_mask on cuda now



/data/lige/miniconda3/envs/HKNet/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using: cuda:0
Using seed 1234.
Dataset: wisconsin
Num classes: 5
dict_keys(['adj_train', 'features', 'labels', 'idx_train', 'idx_val', 'idx_test', 'adj_train_norm'])
(251, 251)
torch.Size([251, 1703])


/data/lige/HKN/utils/data_utils.py:347: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj = nx.adjacency_matrix(G, sorted(G.nodes()))


NCModel(
  (encoder): BMLP(
    (layers): Sequential(
      (0): BMLP(
        (linear): BLinear(in_features=1703, out_features=64, c=tensor([1.], device='cuda:0'), use_bias=1, act=<function relu at 0x7fb635f3f370>)
      )
    )
  )
  (decoder): PoincareDecoder()
)
Total number of parameters: 109445


In [2]:
if config_args['model_config']['model'][0] == 'BMLP':
    for name, param in model.named_parameters():
        if param.requires_grad:
            print(name, param.data)
    print(model.encode(data['features'], data['adj_train_norm']))#[2]
    for name, param in model.named_parameters():
        print(name, param.requires_grad)  # 确保所有参数的 requires_grad 都是 True
elif config_args['model_config']['model'][0] == 'BKNet':
    for name, param in model.named_parameters():
        if param.requires_grad:
            print(name, param.data)
    print(model.encode(data['features'], (nei, nei_mask)))#[2]
    for name, param in model.named_parameters():
        print(name, param.requires_grad)  # 确保所有参数的 requires_grad 都是 True


encoder.layers.0.linear.weight tensor([[-0.0442, -0.0694, -0.0055,  ...,  0.0167,  0.0654,  0.0218],
        [ 0.0190,  0.0472,  0.0733,  ..., -0.0487,  0.0387,  0.0355],
        [-0.0693,  0.0475, -0.0709,  ..., -0.0135, -0.0324,  0.0420],
        ...,
        [-0.0812, -0.0316,  0.0350,  ..., -0.0817, -0.0659, -0.0149],
        [ 0.0214,  0.0800,  0.0784,  ...,  0.0245,  0.0677,  0.0106],
        [ 0.0500,  0.0287,  0.0559,  ..., -0.0123,  0.0511, -0.0228]],
       device='cuda:0', dtype=torch.float64)
encoder.layers.0.linear.bias tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       device='cuda:0', dtype=torch.float64)
decoder.cls tensor([[-0.0231, -0.0347,  0.0525, -0.1259,  0.0105, -0.1135, -0.0525, -0.1153,
         -0.0316, -0.2283, -0.0035, -0.158

### See Training Details

In [3]:
decoded_values=[]
embedded_values=[]
model_linear_weight=[]

# 检查权重更新
def check_weights(model, epoch):
    print(f"Epoch {epoch + 1} - Model Linear Weight:")
    for name, param in model.named_parameters():
        if 'weight' in name:
            print(f"{name}: {param.data}")

for epoch in range(args.epochs):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    if args.model == 'HKPNet':
        embeddings = model.encode(data['features'], (nei, nei_mask))
    elif args.model == 'BKNet':
        embeddings = model.encode(data['features'], (nei, nei_mask))
    else:
        embeddings = model.encode(data['features'], data['adj_train_norm'])

    idx=data[f'idx_train']
    output=model.decode(embeddings, data['adj_train_norm'], idx)
    train_metrics = model.compute_metrics(embeddings, data, 'train')
    
    # 检查 decoded_values 和 embeddings
    embedded_values.append(embeddings)
    #idx = data[f'idx_train']
    #output = model.decode(embeddings, data['adj_train_norm'], idx)
    decoded_values.append(output)
    model_linear_weight.append(model.encoder.layers[0].linear.weight.clone())
    
    # 检查梯度是否被正确计算
    #train_metrics['loss'].backward()
    loss = F.cross_entropy(output, data['labels'][idx])
    loss.backward()

    for name, param in model.named_parameters():
        if param.grad is not None:
            print(f"Grad of {name}: {param.grad.abs().mean()}")
        else:
            print(f"Grad of {name}: None")  # 检查梯度是否为 None
    
    if args.grad_clip is not None:
        torch.nn.utils.clip_grad_norm_(model.parameters(), args.grad_clip)
    optimizer.step()
    lr_scheduler.step()
    
    # 打印和检查权重是否被更新
    check_weights(model, epoch)
    
    if (epoch + 1) % args.log_freq == 0:
        print(" ".join([
            'Epoch: {:04d}'.format(epoch + 1),
            'lr: {}'.format(lr_scheduler.get_last_lr()),
            format_metrics(train_metrics, 'train'),
            'time: {:.4f}s'.format(time.time() - t)
        ]))
    
    with torch.no_grad():
        if (epoch + 1) % args.eval_freq == 0:
            model.eval()
            if args.model == 'HKPNet':
                embeddings = model.encode(data['features'], (nei, nei_mask))
            elif args.model == 'BKNet':
                embeddings = model.encode(data['features'], (nei, nei_mask))
            else:
                embeddings = model.encode(data['features'],
                                        data['adj_train_norm'])
            val_metrics = model.compute_metrics(embeddings, data, 'val')
            if (epoch + 1) % args.log_freq == 0:
                print(" ".join([
                    'Epoch: {:04d}'.format(epoch + 1),
                    format_metrics(val_metrics, 'val')
                ]))
            if model.has_improved(best_val_metrics, val_metrics):
                best_test_metrics = model.compute_metrics(
                    embeddings, data, 'test')
                best_emb = embeddings.cpu()
                if args.save:
                    np.save(os.path.join(save_dir, 'embeddings.npy'),
                            best_emb.detach().numpy())
                best_val_metrics = val_metrics
                counter = 0
            else:
                counter += 1
                if counter == args.patience and epoch > args.min_epochs:
                    print("Early stopping")
                    break

print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
if not best_test_metrics:
    model.eval()
    best_emb = model.encode(data['features'], data['adj_train_norm'])
    best_test_metrics = model.compute_metrics(best_emb, data, 'test')
print(" ".join(
    ["Val set results:",
    format_metrics(best_val_metrics, 'val')]))
print(" ".join(
    ["Test set results:",
    format_metrics(best_test_metrics, 'test')]))
if args.save:
    np.save(os.path.join(save_dir, 'embeddings.npy'),
            best_emb.cpu().detach().numpy())
    if hasattr(model.encoder, 'att_adj'):
        filename = os.path.join(save_dir, args.dataset + '_att_adj.p')
        pickle.dump(model.encoder.att_adj.cpu().to_dense(),
                    open(filename, 'wb'))
        print('Dumped attention adj: ' + filename)

    torch.save(model.state_dict(), os.path.join(save_dir, 'model.pth'))
    json.dump(vars(args), open(os.path.join(save_dir, 'config.json'), 'w'))
    logging.info(f"Saved model in {save_dir}")


Grad of encoder.layers.0.linear.weight: 0.0
Grad of encoder.layers.0.linear.bias: 0.0
Grad of decoder.origin: None
Grad of decoder.cls: 0.06252067294227559
Grad of decoder.bias: 0.13273885846138
Epoch 1 - Model Linear Weight:
encoder.layers.0.linear.weight: tensor([[-0.0442, -0.0694, -0.0055,  ...,  0.0167,  0.0654,  0.0218],
        [ 0.0190,  0.0472,  0.0733,  ..., -0.0487,  0.0387,  0.0355],
        [-0.0693,  0.0475, -0.0709,  ..., -0.0135, -0.0324,  0.0420],
        ...,
        [-0.0812, -0.0316,  0.0350,  ..., -0.0817, -0.0659, -0.0149],
        [ 0.0214,  0.0800,  0.0784,  ...,  0.0245,  0.0677,  0.0106],
        [ 0.0500,  0.0287,  0.0559,  ..., -0.0123,  0.0511, -0.0228]],
       device='cuda:0', dtype=torch.float64)
Grad of encoder.layers.0.linear.weight: 0.0
Grad of encoder.layers.0.linear.bias: 0.0
Grad of decoder.origin: None
Grad of decoder.cls: 0.06232797900179261
Grad of decoder.bias: 0.13234056532382965
Epoch 2 - Model Linear Weight:
encoder.layers.0.linear.weight: te

KeyboardInterrupt: 

In [12]:
for epoch in range(args.epochs):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    if args.model == 'HKPNet':
        embeddings = model.encode(data['features'], (nei, nei_mask))
    elif args.model == 'BKNet':
        embeddings = model.encode(data['features'], (nei, nei_mask))
    else:
        embeddings = model.encode(data['features'], data['adj_train_norm'])

    #idx=data[f'idx_train']
    #output=model.decode(embeddings, data['adj_train_norm'], idx)
    train_metrics = model.compute_metrics(embeddings, data, 'train')
    
    # 检查 decoded_values 和 embeddings
    #embedded_values.append(embeddings)
    #idx = data[f'idx_train']
    #output = model.decode(embeddings, data['adj_train_norm'], idx)
    #decoded_values.append(output)
    #model_linear_weight.append(model.encoder.layers[0].linear.E_linear.weight.clone())
    
    # 检查梯度是否被正确计算
    train_metrics['loss'].backward()
    #loss = F.cross_entropy(output, data['labels'][idx])
    #loss.backward()

    #for name, param in model.named_parameters():
        #if param.grad is not None:
            #print(f"Grad of {name}: {param.grad.abs().mean()}")
        #else:
            #print(f"Grad of {name}: None")  # 检查梯度是否为 None
    
    if args.grad_clip is not None:
        torch.nn.utils.clip_grad_norm_(model.parameters(), args.grad_clip)
        
    optimizer.step()
    lr_scheduler.step()
    
    # 打印和检查权重是否被更新
    #check_weights(model, epoch)
    
    if (epoch + 1) % args.log_freq == 0:
        print(" ".join([
            'Epoch: {:04d}'.format(epoch + 1),
            'lr: {}'.format(lr_scheduler.get_last_lr()),
            format_metrics(train_metrics, 'train'),
            'time: {:.4f}s'.format(time.time() - t)
        ]))
    
    with torch.no_grad():
        if (epoch + 1) % args.eval_freq == 0:
            model.eval()
            if args.model == 'HKPNet':
                embeddings = model.encode(data['features'], (nei, nei_mask))
            elif args.model == 'BKNet':
                embeddings = model.encode(data['features'], (nei, nei_mask))
            else:
                embeddings = model.encode(data['features'],
                                        data['adj_train_norm'])
            val_metrics = model.compute_metrics(embeddings, data, 'val')
            if (epoch + 1) % args.log_freq == 0:
                print(" ".join([
                    'Epoch: {:04d}'.format(epoch + 1),
                    format_metrics(val_metrics, 'val')
                ]))
            if model.has_improved(best_val_metrics, val_metrics):
                best_test_metrics = model.compute_metrics(
                    embeddings, data, 'test')
                best_emb = embeddings.cpu()
                if args.save:
                    np.save(os.path.join(save_dir, 'embeddings.npy'),
                            best_emb.detach().numpy())
                best_val_metrics = val_metrics
                counter = 0
            else:
                counter += 1
                if counter == args.patience and epoch > args.min_epochs:
                    print("Early stopping")
                    break

print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
if not best_test_metrics:
    model.eval()
    best_emb = model.encode(data['features'], data['adj_train_norm'])
    best_test_metrics = model.compute_metrics(best_emb, data, 'test')
print(" ".join(
    ["Val set results:",
    format_metrics(best_val_metrics, 'val')]))
print(" ".join(
    ["Test set results:",
    format_metrics(best_test_metrics, 'test')]))
if args.save:
    np.save(os.path.join(save_dir, 'embeddings.npy'),
            best_emb.cpu().detach().numpy())
    if hasattr(model.encoder, 'att_adj'):
        filename = os.path.join(save_dir, args.dataset + '_att_adj.p')
        pickle.dump(model.encoder.att_adj.cpu().to_dense(),
                    open(filename, 'wb'))
        print('Dumped attention adj: ' + filename)

    torch.save(model.state_dict(), os.path.join(save_dir, 'model.pth'))
    json.dump(vars(args), open(os.path.join(save_dir, 'config.json'), 'w'))
    logging.info(f"Saved model in {save_dir}")


Epoch: 0005 lr: [5e-05, 5e-05] train_loss: 1.335661 train_acc: 0.531847 train_f1: 0.531847 time: 0.0131s
Epoch: 0005 val_loss: 1.218821 val_acc: 0.592593 val_f1: 0.592593
Epoch: 0010 lr: [5e-05, 5e-05] train_loss: 1.257049 train_acc: 0.531847 train_f1: 0.531847 time: 0.0140s
Epoch: 0010 val_loss: 1.123010 val_acc: 0.592593 val_f1: 0.592593
Epoch: 0015 lr: [5e-05, 5e-05] train_loss: 1.257084 train_acc: 0.531847 train_f1: 0.531847 time: 0.0137s
Epoch: 0015 val_loss: 1.131466 val_acc: 0.592593 val_f1: 0.592593
Epoch: 0020 lr: [5e-05, 5e-05] train_loss: 1.258307 train_acc: 0.531847 train_f1: 0.531847 time: 0.0128s
Epoch: 0020 val_loss: 1.121650 val_acc: 0.592593 val_f1: 0.592593
Epoch: 0025 lr: [5e-05, 5e-05] train_loss: 1.227934 train_acc: 0.531847 train_f1: 0.531847 time: 0.0123s
Epoch: 0025 val_loss: 1.106207 val_acc: 0.592593 val_f1: 0.592593
Epoch: 0030 lr: [5e-05, 5e-05] train_loss: 1.241896 train_acc: 0.531847 train_f1: 0.531847 time: 0.0120s
Epoch: 0030 val_loss: 1.114007 val_acc: 

## BLinear-v2

In [4]:
import argparse
from types import SimpleNamespace
import sys
sys.path.append('/data/lige/HKN')# Please change accordingly!

from __future__ import division
from __future__ import print_function

from geoopt import ManifoldParameter as geoopt_ManifoldParameter
from manifolds.base import ManifoldParameter as base_ManifoldParameter

import datetime
import json
import logging
from optim import RiemannianAdam, RiemannianSGD
import os
import pickle
import time

import numpy as np
import torch
from config import parser
from models.base_models import NCModel, LPModel, GCModel
from utils.data_utils import load_data, get_nei, GCDataset, split_batch
from utils.train_utils import get_dir_name, format_metrics
from utils.eval_utils import acc_f1

from geoopt import ManifoldParameter as geoopt_ManifoldParameter
from manifolds.base import ManifoldParameter as base_ManifoldParameter

import torch.nn.functional as F


config_args = {
    'training_config': {
        'lr': (1e-4, 'learning rate'),
        'dropout': (0.25, 'dropout probability'),
        'cuda': (0, 'which cuda device to use (-1 for cpu training)'),
        'epochs': (1000, 'maximum number of epochs to train for'),
        'weight_decay': (1e-3, 'l2 regularization strength'),
        'optimizer': ('radam', 'which optimizer to use, can be any of [rsgd, radam]'),
        'momentum': (0.999, 'momentum in optimizer'),
        'patience': (15, 'patience for early stopping'),
        'seed': (1234, 'seed for training'),
        'log_freq': (5, 'how often to compute print train/val metrics (in epochs)'),
        'eval_freq': (1, 'how often to compute val metrics (in epochs)'),
        'save': (0, '1 to save model and logs and 0 otherwise'),
        'save_dir': (None, 'path to save training logs and model weights (defaults to logs/task/date/run/)'),
        'sweep_c': (0, ''),
        'lr_reduce_freq': (None, 'reduce lr every lr-reduce-freq or None to keep lr constant'),
        'gamma': (0.5, 'gamma for lr scheduler'),
        'print_epoch': (True, ''),
        'grad_clip': (None, 'max norm for gradient clipping, or None for no gradient clipping'),
        'min_epochs': (300, 'do not early stop before min-epochs')
    },
    'model_config': {
        'use_geoopt': (False, "which manifold class to use, if false then use basd.manifold"),
        'AggKlein':(False, "if false, then use hyperboloid centorid for aggregation"),
        'corr': (0,'0: d(x_i ominus x, x_k), 1: d(x_ik,x_k)'),
        'task': ('nc', 'which tasks to train on, can be any of [lp, nc]'),
        'model': ('BMLP', 'which encoder to use, can be any of [Shallow, MLP, HNN, GCN, GAT, HyperGCN, HyboNet,BKNet,BMLP]'),
        'dim': (64, 'embedding dimension'),
        'manifold': ('PoincareBall', 'which manifold to use, can be any of [Euclidean, Hyperboloid, PoincareBall, Lorentz]'),
        'c': (1.0, 'hyperbolic radius, set to None for trainable curvature'),
        'r': (2., 'fermi-dirac decoder parameter for lp'),
        't': (1., 'fermi-dirac decoder parameter for lp'),
        'margin': (2., 'margin of MarginLoss'),
        'pretrained_embeddings': (None, 'path to pretrained embeddings (.npy file) for Shallow node classification'),
        'pos_weight': (0, 'whether to upweight positive class in node classification tasks'),
        'num_layers': (4, 'number of hidden layers in encoder'),
        'bias': (1, 'whether to use bias (1) or not (0)'),
        'act': ('relu', 'which activation function to use (or None for no activation)'),
        'n_heads': (4, 'number of attention heads for graph attention networks, must be a divisor dim'),
        'alpha': (0.2, 'alpha for leakyrelu in graph attention networks'),
        'double_precision': ('1', 'whether to use double precision'),
        'use_att': (0, 'whether to use hyperbolic attention or not'),
        'local_agg': (0, 'whether to local tangent space aggregation or not'),
        'kernel_size': (8, 'number of kernels'),
        'KP_extent': (0.66, 'influence radius of each kernel point'),
        'radius': (1, 'radius used for kernel point init'),
        'deformable': (False, 'deformable kernel'),
        'linear_before': (64, 'dim of linear before gcn')#64
    },
    'data_config': {
        'dataset': ('wisconsin', 'which dataset to use(cornell,wisconsin,squirrel,cora)'),
        'batch_size': (32, 'batch size for gc'),
        'val_prop': (0.05, 'proportion of validation edges for link prediction'),
        'test_prop': (0.1, 'proportion of test edges for link prediction'),
        'use_feats': (1, 'whether to use node features or not'),
        'normalize_feats': (1, 'whether to normalize input node features'),
        'normalize_adj': (1, 'whether to row-normalize the adjacency matrix'),
        'split_seed': (1234, 'seed for data splits (train/test/val)'),
        'split_graph': (False, 'whether to split the graph')
    }
}

# 将所有参数转换为 SimpleNamespace
args = SimpleNamespace(
    **{k: v[0] for config in config_args.values() for k, v in config.items()}
)

#choose which manifold class to follow 
if args.use_geoopt == False:
    ManifoldParameter = base_ManifoldParameter
else:
    ManifoldParameter = geoopt_ManifoldParameter
np.random.seed(args.seed)#args.seed
torch.manual_seed(args.seed)#args.seed
if int(args.cuda):#args.double_precision
    torch.set_default_dtype(torch.float64)
if int(args.cuda) >= 0:#args.cuda
    torch.cuda.manual_seed(args.seed)#args.seed
args.device = 'cuda:' + str(args.cuda) if int(args.cuda) >= 0 else 'cpu' #args.device actually,<-args.cuda
args.patience = args.epochs if not args.patience else args.patience #args.patience<-args.epochs|args.patience

print(f'Using: {args.device}')
print("Using seed {}.".format(args.seed))
print(f"Dataset: {args.dataset}")

# Load data
data = load_data(args, os.path.join('data', args.dataset))
if args.task == 'gc':
    args.n_nodes, args.feat_dim = data['features'][0].shape
else:
    args.n_nodes, args.feat_dim = data['features'].shape
if args.task == 'nc':
    Model = NCModel
    args.n_classes = int(data['labels'].max() + 1)
    args.data = data
    print(f'Num classes: {args.n_classes}')
elif args.task == 'gc':
    Model = GCModel
    args.n_classes = int(data['labels'].max() + 1)
    print(f'Num classes: {args.n_classes}')
else:
    args.nb_false_edges = len(data['train_edges_false'])
    args.nb_edges = len(data['train_edges'])
    if args.task == 'lp':
        Model = LPModel
        args.n_classes = 2

if not args.lr_reduce_freq:
    args.lr_reduce_freq = args.epochs


###A simple check on data
print(data.keys())
print(data['adj_train'].todense().shape)
print(data['features'].shape)
###A simple check on data

# Model and optimizer
model = Model(args)
print(str(model))
no_decay = ['bias', 'scale']
optimizer_grouped_parameters = [{
    'params': [
        p for n, p in model.named_parameters()
        if p.requires_grad and not any(
            nd in n
            for nd in no_decay) and not isinstance(p, ManifoldParameter)
    ],
    'weight_decay':
    args.weight_decay
}, {
    'params': [
        p for n, p in model.named_parameters() if p.requires_grad and any(
            nd in n
            for nd in no_decay) or isinstance(p, ManifoldParameter)
    ],
    'weight_decay':
    0.0
}]
if args.optimizer == 'radam':
    optimizer = RiemannianAdam(params=optimizer_grouped_parameters,
                                lr=args.lr,
                                stabilize=10)
elif args.optimizer == 'rsgd':
    optimizer = RiemannianSGD(params=optimizer_grouped_parameters,
                                lr=args.lr,
                                stabilize=10)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                step_size=int(
                                                args.lr_reduce_freq),
                                                gamma=float(args.gamma))
tot_params = sum([np.prod(p.size()) for p in model.parameters()])
model = model.to(args.device)
for x, val in data.items():
    if torch.is_tensor(data[x]):
        data[x] = data[x].to(args.device)
print(f"Total number of parameters: {tot_params}")

# Train model for nc:
t_total = time.time()
counter = 0
best_val_metrics = model.init_metric_dict()
best_test_metrics = None
best_emb = None
if args.n_classes > 2:
    f1_average = 'micro'
else:
    f1_average = 'binary'

if args.model == 'HKPNet':
    nei, nei_mask = get_nei(data['adj_train'])
    nei = nei.to(args.device)
    nei_mask = nei_mask.to(args.device)
elif args.model == 'BKNet':
    nei, nei_mask = get_nei(data['adj_train'])
    nei = nei.to(args.device)
    nei_mask = nei_mask.to(args.device) #nei/nei_mask on cuda now



Using: cuda:0
Using seed 1234.
Dataset: wisconsin
Num classes: 5
dict_keys(['adj_train', 'features', 'labels', 'idx_train', 'idx_val', 'idx_test', 'adj_train_norm'])
(251, 251)
torch.Size([251, 1703])
NCModel(
  (encoder): BMLP(
    (layers): Sequential(
      (0): BMLP(
        (linear): BLinear(
          in_features=1703, out_features=64, c=tensor([1.], device='cuda:0'), use_bias=1, act=<function relu at 0x7febe03b7370>, dropout_rate=0.25
          (E_linear): Linear(in_features=1703, out_features=64, bias=True)
          (dropout): Dropout(p=0.25, inplace=False)
        )
      )
      (1): BMLP(
        (linear): BLinear(
          in_features=64, out_features=64, c=tensor([1.], device='cuda:0'), use_bias=1, act=<function relu at 0x7febe03b7370>, dropout_rate=0.25
          (E_linear): Linear(in_features=64, out_features=64, bias=True)
          (dropout): Dropout(p=0.25, inplace=False)
        )
      )
      (2): BMLP(
        (linear): BLinear(
          in_features=64, out_fea

/data/lige/HKN/utils/data_utils.py:347: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj = nx.adjacency_matrix(G, sorted(G.nodes()))


In [5]:
if config_args['model_config']['model'][0] == 'BMLP':
    for name, param in model.named_parameters():
        if param.requires_grad:
            print(name, param.data)
    print(model.encode(data['features'], data['adj_train_norm']))#[2]
    for name, param in model.named_parameters():
        print(name, param.requires_grad)  # 确保所有参数的 requires_grad 都是 True
elif config_args['model_config']['model'][0] == 'BKNet':
    for name, param in model.named_parameters():
        if param.requires_grad:
            print(name, param.data)
    print(model.encode(data['features'], (nei, nei_mask)))#[2]
    for name, param in model.named_parameters():
        print(name, param.requires_grad)  # 确保所有参数的 requires_grad 都是 True


encoder.layers.0.linear.E_linear.weight tensor([[ 4.1756e-02, -1.8398e-02,  5.8131e-02,  ..., -7.5607e-02,
         -7.7221e-02,  6.1167e-02],
        [ 6.5736e-02, -4.8146e-02, -6.1437e-02,  ..., -9.6945e-03,
         -7.6701e-02,  4.9474e-05],
        [ 3.6671e-02, -2.2466e-02, -4.6759e-02,  ...,  5.6713e-02,
         -3.2809e-02,  3.4089e-02],
        ...,
        [ 3.2771e-02, -8.7429e-04,  1.9891e-02,  ..., -5.7626e-02,
         -5.3839e-02, -6.5566e-02],
        [-6.9515e-02,  3.3123e-02,  6.7455e-02,  ..., -6.1532e-02,
         -3.4309e-02, -2.0445e-02],
        [-7.1281e-02, -3.0733e-02,  4.4339e-02,  ...,  4.7381e-02,
         -5.6454e-02, -9.1399e-03]], device='cuda:0', dtype=torch.float64)
encoder.layers.0.linear.E_linear.bias tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 

### See Training Details

In [6]:
decoded_values=[]
embedded_values=[]
model_linear_weight=[]

# 检查权重更新
def check_weights(model, epoch):
    print(f"Epoch {epoch + 1} - Model Linear Weight:")
    for name, param in model.named_parameters():
        if 'weight' in name:
            print(f"{name}: {param.data}")

for epoch in range(args.epochs):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    if args.model == 'HKPNet':
        embeddings = model.encode(data['features'], (nei, nei_mask))
    elif args.model == 'BKNet':
        embeddings = model.encode(data['features'], (nei, nei_mask))
    else:
        embeddings = model.encode(data['features'], data['adj_train_norm'])

    idx=data[f'idx_train']
    output=model.decode(embeddings, data['adj_train_norm'], idx)
    train_metrics = model.compute_metrics(embeddings, data, 'train')
    
    # 检查 decoded_values 和 embeddings
    embedded_values.append(embeddings)
    #idx = data[f'idx_train']
    #output = model.decode(embeddings, data['adj_train_norm'], idx)
    decoded_values.append(output)
    model_linear_weight.append(model.encoder.layers[0].linear.E_linear.weight.clone())
    
    # 检查梯度是否被正确计算
    #train_metrics['loss'].backward()
    loss = F.cross_entropy(output, data['labels'][idx])
    loss.backward()

    for name, param in model.named_parameters():
        if param.grad is not None:
            print(f"Grad of {name}: {param.grad.abs().mean()}")
        else:
            print(f"Grad of {name}: None")  # 检查梯度是否为 None
    
    if args.grad_clip is not None:
        torch.nn.utils.clip_grad_norm_(model.parameters(), args.grad_clip)
    optimizer.step()
    lr_scheduler.step()
    
    # 打印和检查权重是否被更新
    check_weights(model, epoch)
    
    if (epoch + 1) % args.log_freq == 0:
        print(" ".join([
            'Epoch: {:04d}'.format(epoch + 1),
            'lr: {}'.format(lr_scheduler.get_last_lr()),
            format_metrics(train_metrics, 'train'),
            'time: {:.4f}s'.format(time.time() - t)
        ]))
    
    with torch.no_grad():
        if (epoch + 1) % args.eval_freq == 0:
            model.eval()
            if args.model == 'HKPNet':
                embeddings = model.encode(data['features'], (nei, nei_mask))
            elif args.model == 'BKNet':
                embeddings = model.encode(data['features'], (nei, nei_mask))
            else:
                embeddings = model.encode(data['features'],
                                        data['adj_train_norm'])
            val_metrics = model.compute_metrics(embeddings, data, 'val')
            if (epoch + 1) % args.log_freq == 0:
                print(" ".join([
                    'Epoch: {:04d}'.format(epoch + 1),
                    format_metrics(val_metrics, 'val')
                ]))
            if model.has_improved(best_val_metrics, val_metrics):
                best_test_metrics = model.compute_metrics(
                    embeddings, data, 'test')
                best_emb = embeddings.cpu()
                if args.save:
                    np.save(os.path.join(save_dir, 'embeddings.npy'),
                            best_emb.detach().numpy())
                best_val_metrics = val_metrics
                counter = 0
            else:
                counter += 1
                if counter == args.patience and epoch > args.min_epochs:
                    print("Early stopping")
                    break

print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
if not best_test_metrics:
    model.eval()
    best_emb = model.encode(data['features'], data['adj_train_norm'])
    best_test_metrics = model.compute_metrics(best_emb, data, 'test')
print(" ".join(
    ["Val set results:",
    format_metrics(best_val_metrics, 'val')]))
print(" ".join(
    ["Test set results:",
    format_metrics(best_test_metrics, 'test')]))
if args.save:
    np.save(os.path.join(save_dir, 'embeddings.npy'),
            best_emb.cpu().detach().numpy())
    if hasattr(model.encoder, 'att_adj'):
        filename = os.path.join(save_dir, args.dataset + '_att_adj.p')
        pickle.dump(model.encoder.att_adj.cpu().to_dense(),
                    open(filename, 'wb'))
        print('Dumped attention adj: ' + filename)

    torch.save(model.state_dict(), os.path.join(save_dir, 'model.pth'))
    json.dump(vars(args), open(os.path.join(save_dir, 'config.json'), 'w'))
    logging.info(f"Saved model in {save_dir}")


Grad of encoder.layers.0.linear.E_linear.weight: 2.5809900782050787e-05
Grad of encoder.layers.0.linear.E_linear.bias: 0.02421751851952242
Grad of encoder.layers.1.linear.E_linear.weight: 9.578160720436458e-05
Grad of encoder.layers.1.linear.E_linear.bias: 0.02555093674321717
Grad of encoder.layers.2.linear.E_linear.weight: 0.00012054014188678102
Grad of encoder.layers.2.linear.E_linear.bias: 0.035253980344086014
Grad of decoder.origin: None
Grad of decoder.cls: 0.063137306730725
Grad of decoder.bias: 0.13256892561912537
Epoch 1 - Model Linear Weight:
encoder.layers.0.linear.E_linear.weight: tensor([[ 4.1756e-02, -1.8398e-02,  5.8131e-02,  ..., -7.5707e-02,
         -7.7121e-02,  6.1067e-02],
        [ 6.5736e-02, -4.8146e-02, -6.1437e-02,  ..., -9.7922e-03,
         -7.6601e-02, -5.0476e-05],
        [ 3.6571e-02, -2.2467e-02, -4.6660e-02,  ...,  5.6614e-02,
         -3.2909e-02,  3.4089e-02],
        ...,
        [ 3.2771e-02, -9.7402e-04,  1.9792e-02,  ..., -5.7526e-02,
         -5.

In [4]:
for epoch in range(args.epochs):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    if args.model == 'HKPNet':
        embeddings = model.encode(data['features'], (nei, nei_mask))
    elif args.model == 'BKNet':
        embeddings = model.encode(data['features'], (nei, nei_mask))
    else:
        embeddings = model.encode(data['features'], data['adj_train_norm'])

    #idx=data[f'idx_train']
    #output=model.decode(embeddings, data['adj_train_norm'], idx)
    train_metrics = model.compute_metrics(embeddings, data, 'train')
    
    # 检查 decoded_values 和 embeddings
    #embedded_values.append(embeddings)
    #idx = data[f'idx_train']
    #output = model.decode(embeddings, data['adj_train_norm'], idx)
    #decoded_values.append(output)
    #model_linear_weight.append(model.encoder.layers[0].linear.E_linear.weight.clone())
    
    # 检查梯度是否被正确计算
    train_metrics['loss'].backward()
    #loss = F.cross_entropy(output, data['labels'][idx])
    #loss.backward()

    #for name, param in model.named_parameters():
        #if param.grad is not None:
            #print(f"Grad of {name}: {param.grad.abs().mean()}")
        #else:
            #print(f"Grad of {name}: None")  # 检查梯度是否为 None
    
    if args.grad_clip is not None:
        torch.nn.utils.clip_grad_norm_(model.parameters(), args.grad_clip)
        
    optimizer.step()
    lr_scheduler.step()
    
    # 打印和检查权重是否被更新
    #check_weights(model, epoch)
    
    if (epoch + 1) % args.log_freq == 0:
        print(" ".join([
            'Epoch: {:04d}'.format(epoch + 1),
            'lr: {}'.format(lr_scheduler.get_last_lr()),
            format_metrics(train_metrics, 'train'),
            'time: {:.4f}s'.format(time.time() - t)
        ]))
    
    with torch.no_grad():
        if (epoch + 1) % args.eval_freq == 0:
            model.eval()
            if args.model == 'HKPNet':
                embeddings = model.encode(data['features'], (nei, nei_mask))
            elif args.model == 'BKNet':
                embeddings = model.encode(data['features'], (nei, nei_mask))
            else:
                embeddings = model.encode(data['features'],
                                        data['adj_train_norm'])
            val_metrics = model.compute_metrics(embeddings, data, 'val')
            if (epoch + 1) % args.log_freq == 0:
                print(" ".join([
                    'Epoch: {:04d}'.format(epoch + 1),
                    format_metrics(val_metrics, 'val')
                ]))
            if model.has_improved(best_val_metrics, val_metrics):
                best_test_metrics = model.compute_metrics(
                    embeddings, data, 'test')
                best_emb = embeddings.cpu()
                if args.save:
                    np.save(os.path.join(save_dir, 'embeddings.npy'),
                            best_emb.detach().numpy())
                best_val_metrics = val_metrics
                counter = 0
            else:
                counter += 1
                if counter == args.patience and epoch > args.min_epochs:
                    print("Early stopping")
                    break

print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
if not best_test_metrics:
    model.eval()
    best_emb = model.encode(data['features'], data['adj_train_norm'])
    best_test_metrics = model.compute_metrics(best_emb, data, 'test')
print(" ".join(
    ["Val set results:",
    format_metrics(best_val_metrics, 'val')]))
print(" ".join(
    ["Test set results:",
    format_metrics(best_test_metrics, 'test')]))
if args.save:
    np.save(os.path.join(save_dir, 'embeddings.npy'),
            best_emb.cpu().detach().numpy())
    if hasattr(model.encoder, 'att_adj'):
        filename = os.path.join(save_dir, args.dataset + '_att_adj.p')
        pickle.dump(model.encoder.att_adj.cpu().to_dense(),
                    open(filename, 'wb'))
        print('Dumped attention adj: ' + filename)

    torch.save(model.state_dict(), os.path.join(save_dir, 'model.pth'))
    json.dump(vars(args), open(os.path.join(save_dir, 'config.json'), 'w'))
    logging.info(f"Saved model in {save_dir}")


Epoch: 0005 lr: [5e-05, 5e-05] train_loss: 1.067529 train_acc: 0.531847 train_f1: 0.531847 time: 0.0115s
Epoch: 0005 val_loss: 0.993680 val_acc: 0.592593 val_f1: 0.592593
Epoch: 0010 lr: [5e-05, 5e-05] train_loss: 1.060791 train_acc: 0.531847 train_f1: 0.531847 time: 0.0073s
Epoch: 0010 val_loss: 0.984395 val_acc: 0.592593 val_f1: 0.592593
Epoch: 0015 lr: [5e-05, 5e-05] train_loss: 1.063553 train_acc: 0.531847 train_f1: 0.531847 time: 0.0081s
Epoch: 0015 val_loss: 0.984270 val_acc: 0.592593 val_f1: 0.592593
Epoch: 0020 lr: [5e-05, 5e-05] train_loss: 1.054629 train_acc: 0.531847 train_f1: 0.531847 time: 0.0074s
Epoch: 0020 val_loss: 0.985793 val_acc: 0.592593 val_f1: 0.592593
Epoch: 0025 lr: [5e-05, 5e-05] train_loss: 1.057709 train_acc: 0.531847 train_f1: 0.531847 time: 0.0080s
Epoch: 0025 val_loss: 0.992272 val_acc: 0.592593 val_f1: 0.592593
Epoch: 0030 lr: [5e-05, 5e-05] train_loss: 1.055495 train_acc: 0.531847 train_f1: 0.531847 time: 0.0087s
Epoch: 0030 val_loss: 0.988252 val_acc: 

# BLinear-v3

In [7]:
import argparse
from types import SimpleNamespace
import sys
sys.path.append('/data/lige/HKN')# Please change accordingly!

from __future__ import division
from __future__ import print_function

from geoopt import ManifoldParameter as geoopt_ManifoldParameter
from manifolds.base import ManifoldParameter as base_ManifoldParameter

import datetime
import json
import logging
from optim import RiemannianAdam, RiemannianSGD
import os
import pickle
import time

import numpy as np
import torch
from config import parser
from models.base_models import NCModel, LPModel, GCModel
from utils.data_utils import load_data, get_nei, GCDataset, split_batch
from utils.train_utils import get_dir_name, format_metrics
from utils.eval_utils import acc_f1

from geoopt import ManifoldParameter as geoopt_ManifoldParameter
from manifolds.base import ManifoldParameter as base_ManifoldParameter

import torch.nn.functional as F


config_args = {
    'training_config': {
        'lr': (1e-4, 'learning rate'),
        'dropout': (0.25, 'dropout probability'),
        'cuda': (0, 'which cuda device to use (-1 for cpu training)'),
        'epochs': (1000, 'maximum number of epochs to train for'),
        'weight_decay': (1e-3, 'l2 regularization strength'),
        'optimizer': ('radam', 'which optimizer to use, can be any of [rsgd, radam]'),
        'momentum': (0.999, 'momentum in optimizer'),
        'patience': (15, 'patience for early stopping'),
        'seed': (1234, 'seed for training'),
        'log_freq': (5, 'how often to compute print train/val metrics (in epochs)'),
        'eval_freq': (1, 'how often to compute val metrics (in epochs)'),
        'save': (0, '1 to save model and logs and 0 otherwise'),
        'save_dir': (None, 'path to save training logs and model weights (defaults to logs/task/date/run/)'),
        'sweep_c': (0, ''),
        'lr_reduce_freq': (None, 'reduce lr every lr-reduce-freq or None to keep lr constant'),
        'gamma': (0.5, 'gamma for lr scheduler'),
        'print_epoch': (True, ''),
        'grad_clip': (None, 'max norm for gradient clipping, or None for no gradient clipping'),
        'min_epochs': (300, 'do not early stop before min-epochs')
    },
    'model_config': {
        'use_geoopt': (False, "which manifold class to use, if false then use basd.manifold"),
        'AggKlein':(False, "if false, then use hyperboloid centorid for aggregation"),
        'corr': (0,'0: d(x_i ominus x, x_k), 1: d(x_ik,x_k)'),
        'task': ('nc', 'which tasks to train on, can be any of [lp, nc]'),
        'model': ('BMLP', 'which encoder to use, can be any of [Shallow, MLP, HNN, GCN, GAT, HyperGCN, HyboNet,BKNet,BMLP]'),
        'dim': (64, 'embedding dimension'),
        'manifold': ('PoincareBall', 'which manifold to use, can be any of [Euclidean, Hyperboloid, PoincareBall, Lorentz]'),
        'c': (1.0, 'hyperbolic radius, set to None for trainable curvature'),
        'r': (2., 'fermi-dirac decoder parameter for lp'),
        't': (1., 'fermi-dirac decoder parameter for lp'),
        'margin': (2., 'margin of MarginLoss'),
        'pretrained_embeddings': (None, 'path to pretrained embeddings (.npy file) for Shallow node classification'),
        'pos_weight': (0, 'whether to upweight positive class in node classification tasks'),
        'num_layers': (3, 'number of hidden layers in encoder'),
        'bias': (1, 'whether to use bias (1) or not (0)'),
        'act': ('relu', 'which activation function to use (or None for no activation)'),
        'n_heads': (4, 'number of attention heads for graph attention networks, must be a divisor dim'),
        'alpha': (0.2, 'alpha for leakyrelu in graph attention networks'),
        'double_precision': ('1', 'whether to use double precision'),
        'use_att': (0, 'whether to use hyperbolic attention or not'),
        'local_agg': (0, 'whether to local tangent space aggregation or not'),
        'kernel_size': (8, 'number of kernels'),
        'KP_extent': (0.66, 'influence radius of each kernel point'),
        'radius': (1, 'radius used for kernel point init'),
        'deformable': (False, 'deformable kernel'),
        'linear_before': (64, 'dim of linear before gcn')#64
    },
    'data_config': {
        'dataset': ('wisconsin', 'which dataset to use(cornell,wisconsin,squirrel,cora)'),
        'batch_size': (32, 'batch size for gc'),
        'val_prop': (0.05, 'proportion of validation edges for link prediction'),
        'test_prop': (0.1, 'proportion of test edges for link prediction'),
        'use_feats': (1, 'whether to use node features or not'),
        'normalize_feats': (1, 'whether to normalize input node features'),
        'normalize_adj': (1, 'whether to row-normalize the adjacency matrix'),
        'split_seed': (1234, 'seed for data splits (train/test/val)'),
        'split_graph': (False, 'whether to split the graph')
    }
}

# 将所有参数转换为 SimpleNamespace
args = SimpleNamespace(
    **{k: v[0] for config in config_args.values() for k, v in config.items()}
)

#choose which manifold class to follow 
if args.use_geoopt == False:
    ManifoldParameter = base_ManifoldParameter
else:
    ManifoldParameter = geoopt_ManifoldParameter
np.random.seed(args.seed)#args.seed
torch.manual_seed(args.seed)#args.seed
if int(args.cuda):#args.double_precision
    torch.set_default_dtype(torch.float64)
if int(args.cuda) >= 0:#args.cuda
    torch.cuda.manual_seed(args.seed)#args.seed
args.device = 'cuda:' + str(args.cuda) if int(args.cuda) >= 0 else 'cpu' #args.device actually,<-args.cuda
args.patience = args.epochs if not args.patience else args.patience #args.patience<-args.epochs|args.patience

print(f'Using: {args.device}')
print("Using seed {}.".format(args.seed))
print(f"Dataset: {args.dataset}")

# Load data
data = load_data(args, os.path.join('data', args.dataset))
if args.task == 'gc':
    args.n_nodes, args.feat_dim = data['features'][0].shape
else:
    args.n_nodes, args.feat_dim = data['features'].shape
if args.task == 'nc':
    Model = NCModel
    args.n_classes = int(data['labels'].max() + 1)
    args.data = data
    print(f'Num classes: {args.n_classes}')
elif args.task == 'gc':
    Model = GCModel
    args.n_classes = int(data['labels'].max() + 1)
    print(f'Num classes: {args.n_classes}')
else:
    args.nb_false_edges = len(data['train_edges_false'])
    args.nb_edges = len(data['train_edges'])
    if args.task == 'lp':
        Model = LPModel
        args.n_classes = 2

if not args.lr_reduce_freq:
    args.lr_reduce_freq = args.epochs


###A simple check on data
print(data.keys())
print(data['adj_train'].todense().shape)
print(data['features'].shape)
###A simple check on data

# Model and optimizer
model = Model(args)
print(str(model))
no_decay = ['bias', 'scale']
optimizer_grouped_parameters = [{
    'params': [
        p for n, p in model.named_parameters()
        if p.requires_grad and not any(
            nd in n
            for nd in no_decay) and not isinstance(p, ManifoldParameter)
    ],
    'weight_decay':
    args.weight_decay
}, {
    'params': [
        p for n, p in model.named_parameters() if p.requires_grad and any(
            nd in n
            for nd in no_decay) or isinstance(p, ManifoldParameter)
    ],
    'weight_decay':
    0.0
}]
if args.optimizer == 'radam':
    optimizer = RiemannianAdam(params=optimizer_grouped_parameters,
                                lr=args.lr,
                                stabilize=10)
elif args.optimizer == 'rsgd':
    optimizer = RiemannianSGD(params=optimizer_grouped_parameters,
                                lr=args.lr,
                                stabilize=10)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                step_size=int(
                                                args.lr_reduce_freq),
                                                gamma=float(args.gamma))
tot_params = sum([np.prod(p.size()) for p in model.parameters()])
model = model.to(args.device)
for x, val in data.items():
    if torch.is_tensor(data[x]):
        data[x] = data[x].to(args.device)
print(f"Total number of parameters: {tot_params}")

# Train model for nc:
t_total = time.time()
counter = 0
best_val_metrics = model.init_metric_dict()
best_test_metrics = None
best_emb = None
if args.n_classes > 2:
    f1_average = 'micro'
else:
    f1_average = 'binary'

if args.model == 'HKPNet':
    nei, nei_mask = get_nei(data['adj_train'])
    nei = nei.to(args.device)
    nei_mask = nei_mask.to(args.device)
elif args.model == 'BKNet':
    nei, nei_mask = get_nei(data['adj_train'])
    nei = nei.to(args.device)
    nei_mask = nei_mask.to(args.device) #nei/nei_mask on cuda now



Using: cuda:0
Using seed 1234.
Dataset: wisconsin
Num classes: 5
dict_keys(['adj_train', 'features', 'labels', 'idx_train', 'idx_val', 'idx_test', 'adj_train_norm'])
(251, 251)
torch.Size([251, 1703])
NCModel(
  (encoder): BMLP(
    (layers): Sequential(
      (0): BMLP(
        (linear): BLinear(
          in_features=1703, out_features=64, c=tensor([1.], device='cuda:0'), use_bias=1, act=<function relu at 0x7fbb480e7370>, dropout_rate=0.25
          (E_linear): Linear(in_features=1703, out_features=64, bias=False)
          (dropout): Dropout(p=0.25, inplace=False)
        )
      )
      (1): BMLP(
        (linear): BLinear(
          in_features=64, out_features=64, c=tensor([1.], device='cuda:0'), use_bias=1, act=<function relu at 0x7fbb480e7370>, dropout_rate=0.25
          (E_linear): Linear(in_features=64, out_features=64, bias=False)
          (dropout): Dropout(p=0.25, inplace=False)
        )
      )
    )
  )
  (decoder): PoincareDecoder()
)
Total number of parameters: 1136

/data/lige/HKN/utils/data_utils.py:347: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj = nx.adjacency_matrix(G, sorted(G.nodes()))


In [8]:
if config_args['model_config']['model'][0] == 'BMLP':
    for name, param in model.named_parameters():
        if param.requires_grad:
            print(name, param.data)
    print(model.encode(data['features'], data['adj_train_norm']))#[2]
    for name, param in model.named_parameters():
        print(name, param.requires_grad)  # 确保所有参数的 requires_grad 都是 True
elif config_args['model_config']['model'][0] == 'BKNet':
    for name, param in model.named_parameters():
        if param.requires_grad:
            print(name, param.data)
    print(model.encode(data['features'], (nei, nei_mask)))#[2]
    for name, param in model.named_parameters():
        print(name, param.requires_grad)  # 确保所有参数的 requires_grad 都是 True


encoder.layers.0.linear.bias tensor([ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00, -1.5969e+29,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         1.4013e-45,  0.0000e+00,  5.5914e+26,  0.0000e+00, -1.6892e+29,
         4.5821e-41,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  5.8672e+26,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  2.8026e-45,  0.0000e+00,  1.4013e-45,
         0.0000e+00,  4.2039e-45,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         1.3593e-43,  0.0000e+00,  5.9461e+26,  0.0000e+00, -1.0312e+33,
         4.5821e-41,  1.4013e-45,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00], device='cuda:0',
       dtype=torc

In [9]:
decoded_values=[]
embedded_values=[]
model_linear_weight=[]

# 检查权重更新
def check_weights(model, epoch):
    print(f"Epoch {epoch + 1} - Model Linear Weight:")
    for name, param in model.named_parameters():
        if 'weight' in name:
            print(f"{name}: {param.data}")

for epoch in range(args.epochs):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    if args.model == 'HKPNet':
        embeddings = model.encode(data['features'], (nei, nei_mask))
    elif args.model == 'BKNet':
        embeddings = model.encode(data['features'], (nei, nei_mask))
    else:
        embeddings = model.encode(data['features'], data['adj_train_norm'])

    idx=data[f'idx_train']
    output=model.decode(embeddings, data['adj_train_norm'], idx)
    train_metrics = model.compute_metrics(embeddings, data, 'train')
    
    # 检查 decoded_values 和 embeddings
    embedded_values.append(embeddings)
    #idx = data[f'idx_train']
    #output = model.decode(embeddings, data['adj_train_norm'], idx)
    decoded_values.append(output)
    model_linear_weight.append(model.encoder.layers[0].linear.E_linear.weight.clone())
    
    # 检查梯度是否被正确计算
    #train_metrics['loss'].backward()
    loss = F.cross_entropy(output, data['labels'][idx])
    loss.backward()

    for name, param in model.named_parameters():
        if param.grad is not None:
            print(f"Grad of {name}: {param.grad.abs().mean()}")
        else:
            print(f"Grad of {name}: None")  # 检查梯度是否为 None
    
    if args.grad_clip is not None:
        torch.nn.utils.clip_grad_norm_(model.parameters(), args.grad_clip)
    optimizer.step()
    lr_scheduler.step()
    
    # 打印和检查权重是否被更新
    check_weights(model, epoch)
    
    if (epoch + 1) % args.log_freq == 0:
        print(" ".join([
            'Epoch: {:04d}'.format(epoch + 1),
            'lr: {}'.format(lr_scheduler.get_last_lr()),
            format_metrics(train_metrics, 'train'),
            'time: {:.4f}s'.format(time.time() - t)
        ]))
    
    with torch.no_grad():
        if (epoch + 1) % args.eval_freq == 0:
            model.eval()
            if args.model == 'HKPNet':
                embeddings = model.encode(data['features'], (nei, nei_mask))
            elif args.model == 'BKNet':
                embeddings = model.encode(data['features'], (nei, nei_mask))
            else:
                embeddings = model.encode(data['features'],
                                        data['adj_train_norm'])
            val_metrics = model.compute_metrics(embeddings, data, 'val')
            if (epoch + 1) % args.log_freq == 0:
                print(" ".join([
                    'Epoch: {:04d}'.format(epoch + 1),
                    format_metrics(val_metrics, 'val')
                ]))
            if model.has_improved(best_val_metrics, val_metrics):
                best_test_metrics = model.compute_metrics(
                    embeddings, data, 'test')
                best_emb = embeddings.cpu()
                if args.save:
                    np.save(os.path.join(save_dir, 'embeddings.npy'),
                            best_emb.detach().numpy())
                best_val_metrics = val_metrics
                counter = 0
            else:
                counter += 1
                if counter == args.patience and epoch > args.min_epochs:
                    print("Early stopping")
                    break

print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
if not best_test_metrics:
    model.eval()
    best_emb = model.encode(data['features'], data['adj_train_norm'])
    best_test_metrics = model.compute_metrics(best_emb, data, 'test')
print(" ".join(
    ["Val set results:",
    format_metrics(best_val_metrics, 'val')]))
print(" ".join(
    ["Test set results:",
    format_metrics(best_test_metrics, 'test')]))
if args.save:
    np.save(os.path.join(save_dir, 'embeddings.npy'),
            best_emb.cpu().detach().numpy())
    if hasattr(model.encoder, 'att_adj'):
        filename = os.path.join(save_dir, args.dataset + '_att_adj.p')
        pickle.dump(model.encoder.att_adj.cpu().to_dense(),
                    open(filename, 'wb'))
        print('Dumped attention adj: ' + filename)

    torch.save(model.state_dict(), os.path.join(save_dir, 'model.pth'))
    json.dump(vars(args), open(os.path.join(save_dir, 'config.json'), 'w'))
    logging.info(f"Saved model in {save_dir}")


Grad of encoder.layers.0.linear.bias: 1.362113716231686e-34
Grad of encoder.layers.0.linear.E_linear.weight: 0.0003694165117318967
Grad of encoder.layers.1.linear.bias: 4.5141145204427356e-30
Grad of encoder.layers.1.linear.E_linear.weight: 0.0012928835265282096
Grad of decoder.origin: None
Grad of decoder.cls: 0.061057500051571234
Grad of decoder.bias: 0.13706384599208832
Epoch 1 - Model Linear Weight:
encoder.layers.0.linear.E_linear.weight: tensor([[ 4.2522e-02, -3.4012e-02,  8.2583e-05,  ..., -5.6964e-02,
          7.8031e-02,  1.5730e-02],
        [-3.4359e-02,  2.6750e-02, -7.1842e-02,  ...,  7.3780e-02,
          4.5724e-02,  6.1737e-02],
        [ 5.2674e-02, -8.0568e-02, -6.9172e-02,  ..., -5.9368e-03,
          7.0274e-02,  5.9101e-04],
        ...,
        [ 4.9732e-02, -8.6031e-03, -4.1792e-02,  ...,  4.9367e-02,
         -4.6295e-02, -2.3697e-02],
        [-2.9552e-02,  3.9245e-02,  1.9787e-02,  ...,  6.4086e-02,
         -1.2974e-02, -6.2131e-02],
        [-6.6796e-02,  5